In [2]:
import numpy as np
import pandas as pd

In [3]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using Theano backend.


In [4]:
train = pd.read_csv("Dataset/train.csv")
test = pd.read_csv("Dataset/test.csv")

In [9]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
70425,734680255503,"Oh, it's moved to grand farce. They're running...",0,0,0,0,0,0
36119,376826914160,AfD Wikipedia and public opinion \n\nI got int...,0,0,0,0,0,0
35351,368013352828,"""\nHi. I appreciate your efforts to nominate a...",0,0,0,0,0,0
54788,571768075460,"""\n\n RFC on Hungrywolf \n\nNatalie,\n\nSince ...",0,0,0,0,0,0
31465,327473263550,Here is an article about a similar subject: t...,0,0,0,0,0,0


In [5]:
max_features = 20000
maxlen = 100

In [6]:
train = train.sample(frac=1)

In [10]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
70425,734680255503,"Oh, it's moved to grand farce. They're running...",0,0,0,0,0,0
36119,376826914160,AfD Wikipedia and public opinion \n\nI got int...,0,0,0,0,0,0
35351,368013352828,"""\nHi. I appreciate your efforts to nominate a...",0,0,0,0,0,0
54788,571768075460,"""\n\n RFC on Hungrywolf \n\nNatalie,\n\nSince ...",0,0,0,0,0,0
31465,327473263550,Here is an article about a similar subject: t...,0,0,0,0,0,0


In [18]:
list_sentences_train = train["comment_text"].fillna("CVxTz").values

In [19]:
list_sentences_train[:10]

array([ "Oh, it's moved to grand farce. They're running all over the place, trying to ruin my reputation. It's pathetic and they're free to make fools of themselves for all I care, just so long as they stay the hell away from my user page, because it's mine.  Do me a favor; take a look at Ayn Rand and decide for yourself if it needs further changes.",
       "AfD Wikipedia and public opinion \n\nI got into a spat with the girl who AfD'd the article; I'm pretty sure that has something to do with it showing up on AfD. This should be fun.",
       '"\nHi. I appreciate your efforts to nominate a worthy candidate for adminship, and agree that the process can be confusing. Simply put,  has a nomination page at Wikipedia:Requests for adminship/Alabamaboy. That\'s where he needs to accept/answer questions. Once that is done, all you need to do is post the following on the main RfA page:\n{{Wikipedia:Requests for adminship/Alabamaboy}}\nLet me know if you have any questions!  talk "',
       '"

In [ ]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

In [20]:
y[:10]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0]])

In [12]:
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [15]:
tokenizer = text.Tokenizer(num_words=max_features)

In [21]:
tokenizer.fit_on_texts(list(list_sentences_train))

In [35]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 'for': 12,
 'this': 13,
 'not': 14,
 'on': 15,
 'be': 16,
 'as': 17,
 'are': 18,
 'have': 19,
 'your': 20,
 'with': 21,
 'if': 22,
 'article': 23,
 'was': 24,
 'or': 25,
 'but': 26,
 'page': 27,
 'wikipedia': 28,
 'my': 29,
 'an': 30,
 'from': 31,
 'by': 32,
 'do': 33,
 'at': 34,
 'me': 35,
 'about': 36,
 'talk': 37,
 'so': 38,
 'can': 39,
 'what': 40,
 'there': 41,
 'all': 42,
 'has': 43,
 'will': 44,
 'no': 45,
 'please': 46,
 'would': 47,
 'one': 48,
 'like': 49,
 'just': 50,
 'they': 51,
 'he': 52,
 'which': 53,
 'been': 54,
 'any': 55,
 'should': 56,
 'more': 57,
 'we': 58,
 "don't": 59,
 'other': 60,
 'some': 61,
 'who': 62,
 'see': 63,
 'here': 64,
 'also': 65,
 'his': 66,
 'think': 67,
 'because': 68,
 'know': 69,
 'how': 70,
 'edit': 71,
 'am': 72,
 "i'm": 73,
 'why': 74,
 'people': 75,
 'only': 76,
 "it's": 77,
 'up': 78,
 'out': 79,
 'use': 80,
 'articles': 81,

In [22]:
list_sentences_train[:10]

array([ "Oh, it's moved to grand farce. They're running all over the place, trying to ruin my reputation. It's pathetic and they're free to make fools of themselves for all I care, just so long as they stay the hell away from my user page, because it's mine.  Do me a favor; take a look at Ayn Rand and decide for yourself if it needs further changes.",
       "AfD Wikipedia and public opinion \n\nI got into a spat with the girl who AfD'd the article; I'm pretty sure that has something to do with it showing up on AfD. This should be fun.",
       '"\nHi. I appreciate your efforts to nominate a worthy candidate for adminship, and agree that the process can be confusing. Simply put,  has a nomination page at Wikipedia:Requests for adminship/Alabamaboy. That\'s where he needs to accept/answer questions. Once that is done, all you need to do is post the following on the main RfA page:\n{{Wikipedia:Requests for adminship/Alabamaboy}}\nLet me know if you have any questions!  talk "',
       '"

In [23]:
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [25]:
list_tokenized_train[:10]

[[295,
  77,
  755,
  2,
  2093,
  9195,
  897,
  1632,
  42,
  156,
  1,
  194,
  253,
  2,
  4743,
  29,
  2535,
  77,
  1756,
  4,
  897,
  188,
  2,
  97,
  5704,
  3,
  737,
  12,
  42,
  7,
  473,
  50,
  38,
  259,
  17,
  51,
  582,
  1,
  851,
  522,
  31,
  29,
  90,
  27,
  68,
  77,
  1090,
  33,
  35,
  5,
  1862,
  153,
  5,
  152,
  34,
  15448,
  9949,
  4,
  743,
  12,
  195,
  22,
  11,
  355,
  388,
  418],
 [842,
  28,
  4,
  513,
  320,
  7,
  293,
  140,
  5,
  14246,
  21,
  1,
  1991,
  62,
  1,
  23,
  73,
  568,
  196,
  9,
  43,
  168,
  2,
  33,
  21,
  11,
  1272,
  78,
  15,
  842,
  13,
  56,
  16,
  1131],
 [174,
  7,
  657,
  20,
  1564,
  2,
  2673,
  5,
  2309,
  2685,
  12,
  2837,
  4,
  263,
  9,
  1,
  575,
  39,
  16,
  1931,
  318,
  206,
  43,
  5,
  1130,
  27,
  34,
  28,
  1236,
  12,
  2837,
  210,
  128,
  52,
  355,
  2,
  859,
  501,
  232,
  393,
  9,
  8,
  201,
  42,
  6,
  123,
  2,
  33,
  8,
  291,
  1,
  425,
  15,
  1,
  437,
  1

In [26]:
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [36]:
X_t[:10]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,   295,    77,
          755,     2,  2093,  9195,   897,  1632,    42,   156,     1,
          194,   253,     2,  4743,    29,  2535,    77,  1756,     4,
          897,   188,     2,    97,  5704,     3,   737,    12,    42,
            7,   473,    50,    38,   259,    17,    51,   582,     1,
          851,   522,    31,    29,    90,    27,    68,    77,  1090,
           33,    35,     5,  1862,   153,     5,   152,    34, 15448,
         9949,     4,   743,    12,   195,    22,    11,   355,   388,
          418],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

In [46]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences = True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation = "relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs = inp, outputs=x)
    model.compile(loss='binary_crossentropy', 
                  optimizer = "adam",
                  metrics = ['accuracy'])
    return model

In [47]:
model = get_model()

In [48]:
batch_size = 32
epochs = 10

In [49]:
file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

In [50]:
callbacks_list = [checkpoint, early]

In [51]:
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

Train on 86265 samples, validate on 9586 samples
Epoch 1/10
86265/86265 [==============================] - 326s - loss: 0.0718 - acc: 0.9771 - val_loss: 0.0495 - val_acc: 0.9823
Epoch 2/10
86265/86265 [==============================] - 335s - loss: 0.0467 - acc: 0.9827 - val_loss: 0.0491 - val_acc: 0.9826
Epoch 3/10
86265/86265 [==============================] - 335s - loss: 0.0401 - acc: 0.9846 - val_loss: 0.0497 - val_acc: 0.9826
Epoch 4/10
86265/86265 [==============================] - 341s - loss: 0.0340 - acc: 0.9865 - val_loss: 0.0543 - val_acc: 0.9818
Epoch 5/10
86265/86265 [==============================] - 359s - loss: 0.0289 - acc: 0.9884 - val_loss: 0.0624 - val_acc: 0.9817
Epoch 6/10
86265/86265 [==============================] - 381s - loss: 0.0242 - acc: 0.9902 - val_loss: 0.0694 - val_acc: 0.9812
Epoch 7/10
86265/86265 [==============================] - 406s - loss: 0.0201 - acc: 0.9919 - val_loss: 0.0779 - val_acc: 0.9813
Epoch 8/10
86265/86265 [========================

In [52]:
model.load_weights(file_path)

In [53]:
y_test = model.predict(X_te)

In [54]:
sample_submission = pd.read_csv("Dataset/sample_submission.csv")

In [55]:
sample_submission[list_classes] = y_test

In [57]:
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.002309,6.833647e-06,0.000619,7.677724e-05,0.000352,0.000224
1,6102620,0.000202,8.016092e-08,0.000051,1.333412e-06,0.000020,0.000010
2,14563293,0.000056,2.375857e-08,0.000019,3.501007e-07,0.000005,0.000003
3,21086297,0.004250,7.620432e-06,0.001038,1.040905e-04,0.000647,0.000255
4,22982444,0.002485,5.278134e-06,0.000584,7.248139e-05,0.000375,0.000192


In [56]:
sample_submission.to_csv("baseline_LSTM.csv", index=False)